$$
    \tau = J \dot{\omega} + \omega \times (J \omega) \\
    \tau = I J \dot{\omega} + \hat{\omega} J \omega \\
    \tau = (\dot{\omega}^T \otimes I) \text{vec}(J) + (\hat{\omega} \otimes \omega) \text{vec}(J) \\
    \tau = (\dot{\omega}^T \otimes I + \hat{\omega} \otimes \omega) \text{vec}(J) \\
$$

The model input is then $x$
$$
    x =
    \left[\begin{array}{cccc}
        v_B^T & \omega_B^T & \Omega^T & \dot{\Omega}^T \\
    \end{array}\right] ^ T\\
    x \in \mathbb{R}^{13}\\
$$

The matrix of coefficients is then defined as $C \in \mathbb{R}^{6 \times 13}$.

$$
    z = \left[\begin{array}{c}
        1 \\
        x \\
        x \otimes x \\
    \end{array}\right] \in \mathbb{R}^{183}
$$
$$
    \hat{\tau} = C z \\
    C \in \mathbb{R}^{3 \times 183}
$$

$$
    \text{vec}(S) = A \, \text{vech}(S) : \forall Sym( \mathbb{R^{3 \times 3}}) \\
    A = \left[\begin{array}{cccccc}
        1 & 0 & 0 & 0 & 0 & 0 \\
        0 & 1 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 1 & 0 & 0 \\
        0 & 1 & 0 & 0 & 0 & 0 \\
        0 & 0 & 1 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 1 & 0 \\
        0 & 0 & 0 & 1 & 0 & 0 \\
        0 & 0 & 0 & 0 & 1 & 0 \\
        0 & 0 & 0 & 0 & 0 & 1 \\
    \end{array}\right]
$$

$$
    \begin{aligned}
        \tau - \hat{\tau} &= \underbrace{(I \otimes \dot{\omega} + \hat{\omega} \otimes \omega) \, A }_{B \in \mathbb{R}^{3 \times 6}} \, \text{vech}(J) - C z \\
        
        &= \left[\begin{array}{cc} B & C \end{array}\right] 
        \left[\begin{array}{c} 
            \text{vech}(J) \\
            z 
        \end{array}\right]\\
    \end{aligned}
$$

$$
    \begin{aligned}
        \tau - \hat{\tau} &= \underbrace{(I \otimes \dot{\omega} + \hat{\omega} \otimes \omega)}_{B \in \mathbb{R}^{3 \times 9}} \text{vec}(J) - C z \\

        &= \left[\begin{array}{cc} B & C \end{array}\right] 
        \left[\begin{array}{c} 
            \text{vec}(J) \\
            z 
        \end{array}\right]\\
    \end{aligned}
$$

In [13]:
x = rand(13)
[1; x; kron(x, x)];

In [1]:
using LinearAlgebra
using Symbolics
using Latexify

In [2]:
# J * ω
# kron(ω', Id) #* vec(J)
# kron(I(3)', ω̇)
# kron(ω̇', Id) * vec(J)

In [14]:
Id = diagm(ones(Int,3))
ω = [1;2;3]
J = diagm([1;2;3])

kron(ω', Id)

3×9 Matrix{Int64}:
 1  0  0  2  0  0  3  0  0
 0  1  0  0  2  0  0  3  0
 0  0  1  0  0  2  0  0  3

In [10]:
vec(J)' * kron(ω', Id)'

1×3 adjoint(::Vector{Int64}) with eltype Int64:
 1  4  9

In [ ]:
Id = diagm(ones(Int,3))
kron(Id, ω̇)

In [55]:
kron(I(3)', ω̇) + kron(hat(ω)', ω)

9×3 Matrix{Num}:
      ω̇[1]   ω[1]*ω[3]  -ω[1]*ω[2]
      ω̇[2]   ω[2]*ω[3]   -(ω[2]^2)
      ω̇[3]      ω[3]^2  -ω[2]*ω[3]
 -ω[1]*ω[3]       ω̇[1]      ω[1]^2
 -ω[2]*ω[3]       ω̇[2]   ω[1]*ω[2]
  -(ω[3]^2)       ω̇[3]   ω[1]*ω[3]
  ω[1]*ω[2]   -(ω[1]^2)       ω̇[1]
     ω[2]^2  -ω[1]*ω[2]       ω̇[2]
  ω[2]*ω[3]  -ω[1]*ω[3]       ω̇[3]

In [15]:
@variables ω[1:3] ω̇[1:3] J[1:3, 1:3]
hat(x) = [0. -x[3] x[2]; x[3] 0. -x[1]; -x[2] x[1] 0.]
# (hat(ω) * J * ω)
# (ω × (J * ω))
vec(J)

9-element reshape(::Symbolics.Arr{Num, 2}, 9) with eltype Num:
 J[1, 1]
 J[2, 1]
 J[3, 1]
 J[1, 2]
 J[2, 2]
 J[3, 2]
 J[1, 3]
 J[2, 3]
 J[3, 3]

In [39]:
function vech(x)
    k = size(x, 1)
    a = zeros(eltype(x), Int((k^2-k)/2 + k))
    m = 1
    for i = 1:k
        for j = 1:i
            a[m] = x[i,j]
            m += 1
        end
    end
    a
end

6-element Vector{Num}:
 u
 v
 x
 w
 y
 z

In [53]:
@variables u v w x y z

test = [u v w; 
        v x y;
        w y z];
A = [1 0 0 0 0 0;
     0 1 0 0 0 0;
     0 0 0 1 0 0;
     0 1 0 0 0 0;
     0 0 1 0 0 0;
     0 0 0 0 1 0;
     0 0 0 1 0 0;
     0 0 0 0 1 0;
     0 0 0 0 0 1];
b = vech(test)

A * vech(test) 

9-element Vector{Num}:
 u
 v
 w
 v
 x
 y
 w
 y
 z

In [56]:
A * vech(test) .== vec(test)

9-element Vector{Num}:
 u == u
 v == v
 w == w
 v == v
 x == x
 y == y
 w == w
 y == y
 z == z

In [58]:
print(latexify(A))

\begin{equation}
\left[
\begin{array}{cccccc}
1 & 0 & 0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 \\
\end{array}
\right]
\end{equation}


In [ ]:
A = [1 0 0;
     0 1 0;
     0 1 0;
     0 0 1];
